### **MALIS Project: Image Captcha simulation and labeling using Machine Learning**


---




Deepika SIVASANKARAN, Abhishek BURSE, Divya Teresa JOHNSON

In [3]:
!nvidia-smi

Thu Jan 20 17:48:29 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Importing the libraries

In [4]:
import tensorflow as tf
import cv2
import glob
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionResNetV2, MobileNet, ResNet101, InceptionV3, VGG19, ResNet152, NASNetLarge, ResNet152V2
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_predict, cross_val_score
from collections import Counter
import matplotlib.pyplot as plt



In [ ]:
!python --version

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!unzip '/content/drive/MyDrive/cptcha.zip' -d '/content/drive/MyDrive'

### Initialising arrays to contain the data

In [9]:
class_label=sorted(os.listdir('/content/drive/MyDrive/cptcha'))
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
X=[]
Y=[]

In [10]:
print(class_label)

['airplane', 'alarm_clock', 'bicycle', 'car', 'chair', 'fire_hydrant', 'flower', 'mushroom', 'sailboat', 'umbrella']


### Append images to arrays

In [11]:
def prepareNameWithLabels():
    for i in class_label:
        sourceFiles=os.listdir('/content/drive/MyDrive/cptcha/'+i)
        for val in sourceFiles:
            img = cv2.imread('/content/drive/MyDrive/cptcha/'+i+'/'+val, cv2.IMREAD_COLOR)
            
            X.append(img)
            for j in range(len(class_label)):
              if(i==class_label[j]):
                  Y.append(j)
                    
prepareNameWithLabels()

X = np.asarray(X)
Y = np.asarray(Y)
print(Y)
print(X.shape, Y.shape)



[0 0 0 ... 9 9 9]
(20351, 100, 100, 3) (20351,)


### Split to testing,training and validation images

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, stratify=Y_train)
print(sorted(Counter(Y_train)))
print(Counter(Y_test))
print(Counter(Y_val))
le.fit(class_label)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Counter({6: 519, 0: 444, 3: 418, 2: 405, 5: 400, 9: 394, 1: 388, 7: 377, 4: 373, 8: 353})
Counter({6: 208, 0: 178, 3: 167, 2: 162, 5: 160, 9: 158, 1: 155, 7: 150, 4: 149, 8: 141})


LabelEncoder()

### Extract features using the models

In [13]:

model = ResNet50(weights='imagenet',include_top=False,input_shape=(100,100,3))

feature_extractor = model.predict(X_train)
shape =feature_extractor.shape
feature = feature_extractor.reshape(feature_extractor.shape[0],-1)

test_features_extractor =model.predict(X_val)
test_feature = test_features_extractor.reshape(test_features_extractor.shape[0],-1)

### Random Forest Classifier

In [14]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#RandomForest Classifier
parameters = {'n_estimators':[1, 10, 50, 70, 100, 200, 1000]}
clf = RandomForestClassifier()
clf = GridSearchCV(clf, parameters)
clf.fit(feature,Y_train)

predicted_output = clf.predict(test_feature)
from sklearn.metrics import classification_report
report = classification_report(Y_val, predicted_output)
print(report)


              precision    recall  f1-score   support

           0       0.86      0.84      0.85       178
           1       0.78      0.87      0.82       155
           2       0.70      0.54      0.61       162
           3       0.79      0.78      0.78       167
           4       0.63      0.64      0.63       149
           5       0.56      0.51      0.53       160
           6       0.79      0.85      0.82       208
           7       0.74      0.79      0.76       150
           8       0.64      0.72      0.67       141
           9       0.50      0.47      0.48       158

    accuracy                           0.71      1628
   macro avg       0.70      0.70      0.70      1628
weighted avg       0.70      0.71      0.70      1628



### K Nearest Neighbours

In [23]:
from sklearn.neighbors import KNeighborsClassifier
#K nearest Neighbours
knn = KNeighborsClassifier(n_neighbors = 6).fit(feature, Y_train)
predicted_output = knn.predict(test_feature)

from sklearn.metrics import classification_report
report = classification_report(Y_val, predicted_output)
print(report)

              precision    recall  f1-score   support

           0       0.73      0.86      0.79       178
           1       0.63      0.59      0.61       155
           2       0.53      0.52      0.53       162
           3       0.75      0.72      0.73       167
           4       0.52      0.44      0.48       149
           5       0.50      0.50      0.50       160
           6       0.61      0.88      0.72       208
           7       0.74      0.56      0.64       150
           8       0.60      0.74      0.66       141
           9       0.51      0.25      0.33       158

    accuracy                           0.62      1628
   macro avg       0.61      0.61      0.60      1628
weighted avg       0.61      0.62      0.61      1628



### Support Vector Machine

In [15]:
from sklearn import svm
svmmodel = svm.SVC(kernel='linear',verbose=True)
svmmodel.fit(feature,Y_train)
#Support Vector Machine
predicted_output = svmmodel.predict(test_feature)

from sklearn.metrics import classification_report
report = classification_report(Y_val, predicted_output)
print('\n',report)



[LibSVM]
               precision    recall  f1-score   support

           0       0.90      0.92      0.91       178
           1       0.77      0.87      0.82       155
           2       0.61      0.60      0.61       162
           3       0.79      0.78      0.79       167
           4       0.66      0.67      0.66       149
           5       0.65      0.69      0.67       160
           6       0.86      0.88      0.87       208
           7       0.86      0.79      0.83       150
           8       0.77      0.74      0.76       141
           9       0.64      0.55      0.59       158

    accuracy                           0.76      1628
   macro avg       0.75      0.75      0.75      1628
weighted avg       0.76      0.76      0.75      1628



### Ensemble (RF + SVM) model

In [20]:
from sklearn.ensemble import VotingClassifier
#Ensemble model

estimators=[('RF', clf), ('SVC', svmmodel)]
ensemble = VotingClassifier(estimators, voting='hard')


ensemble.fit(feature, Y_train)
predicted_output = ensemble.predict(test_feature)

from sklearn.metrics import classification_report
report = classification_report(Y_val, predicted_output)
print('\n',report)

[LibSVM]
               precision    recall  f1-score   support

           0       0.85      0.92      0.88       178
           1       0.68      0.90      0.78       155
           2       0.59      0.65      0.62       162
           3       0.76      0.77      0.77       167
           4       0.61      0.67      0.64       149
           5       0.61      0.59      0.60       160
           6       0.81      0.89      0.85       208
           7       0.89      0.73      0.80       150
           8       0.81      0.71      0.75       141
           9       0.70      0.36      0.47       158

    accuracy                           0.73      1628
   macro avg       0.73      0.72      0.72      1628
weighted avg       0.73      0.73      0.72      1628



### Principal Component Analysis

In [ ]:
#Principal Component Analysis


X=[]
y=[]

def prepareNameWithLabels():
    for i in class_label:
        sourceFiles=os.listdir('/content/drive/MyDrive/cptcha/'+i)
        for val in sourceFiles:
            img = cv2.imread('/content/drive/MyDrive/cptcha/'+i+'/'+val, cv2.IMREAD_COLOR)

            blue,green,red = cv2.split(img) 
            from sklearn.decomposition import PCA
            pca = PCA(10)
 
            
            red_transformed = pca.fit_transform(red)
            red_inverted = pca.inverse_transform(red_transformed)
            
            green_transformed = pca.fit_transform(green)
            green_inverted = pca.inverse_transform(green_transformed)
            
            blue_transformed = pca.fit_transform(blue)
            blue_inverted = pca.inverse_transform(blue_transformed)
            img_compressed = (np.dstack((red_inverted, red_inverted, red_inverted))).astype(np.uint8)
            X.append(img_compressed)
            
            for j in range(len(class_label)):
              if(i==class_label[j]):
                  Y.append(j)
                    
prepareNameWithLabels()

X = np.asarray(X)
Y = np.asarray(Y)
print(Y)
print(X.shape, Y.shape)



### Testing with model

In [ ]:
from matplotlib import pyplot as plt
n=np.random.randint(0, X_test.shape[0])
img = cv2.imread('https://wallup.net/wp-content/uploads/2019/09/716914-cessna-airplane-aircraft-transport.jpg',cv2.IMREAD_COLOR)
from PIL import Image
# importing modules
import urllib.request
import requests

im = Image.open(requests.get('https://wallup.net/wp-content/uploads/2019/09/716914-cessna-airplane-aircraft-transport.jpg', stream=True).raw)
input_img = np.expand_dims(im.resize((100,100)), axis=0)
input_img_feature = model.predict(input_img)
input_img_features = input_img_feature.reshape(input_img_feature.shape[0],-1)
prediction_SVM = svmmodel.predict(input_img_features)[0]
print('Predicted Label:')
print(class_label[prediction_SVM], prediction_SVM)


### Saving the models


In [24]:
import pickle


filename = '/content/drive/MyDrive/models/svmmodel.h5'
pickle.dump(svmmodel, open(filename, 'wb'))

filename = '/content/drive/MyDrive/models/ensemblemodel.h5'
pickle.dump(ensemble, open(filename, 'wb'))

filename = '/content/drive/MyDrive/models/rfmodel.h5'
pickle.dump(clf, open(filename, 'wb'))


filename = '/content/drive/MyDrive/models/knn.h5'
pickle.dump(knn, open(filename, 'wb'))